In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.callbacks import LearningRateMonitor

from dataset import COCODataset
from lightningmodule import LightningNet, GuidedLightningNet
from utils import resize, predict, batch_loc_loss
from model import Resnext50

from captum.attr import Saliency
from captum.attr import visualization as viz\

BATCH_SIZE = 128 if torch.cuda.is_available() else 64
NUM_WORKERS = 2

/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
trainset = COCODataset('./pascalvoc', 'trainval')

# print(os.path.join(top, file) for top, dir, file in os.walk(os.path.join('./pascalvoc', 'VOC' + 'trainval' + '_06-Nov-2007', 'VOCdevkit', 'VOC2007', 'JPEGImages')))

# print(trainset.__getitem__(9))

# print(images)
# print(boxes)

# new_image, new_boxes = resize(images, boxes, (224, 224))

# print(new_image)
# print(new_boxes)

# tf = torchvision.transforms.Compose([
#         torchvision.transforms.ToTensor(),
#         torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
#     ])
# new_image = tf(new_image)

# print(new_image)

# for top, dir, file in os.walk(os.path.join('./pascalvoc', 'VOC' + 'trainval' + '_06-Nov-2007', 'VOCdevkit', 'VOC2007', 'JPEGImages')):
#     images = sorted([os.path.join(top, name) for name in file])

def collate_fn(batch):

   # print(batch[0][2].size())

   images = list()
   boxes = list()
   labels = list()

   for b in batch:
      images.append(b[0])
      labels.append(b[1])
      boxes.append(b[2])

   images = torch.stack(images, dim=0)
   # boxes = torch.stack(boxes, dim=0)
   labels = torch.stack(labels, dim=0)

   return images, labels, boxes 

trainset, valset = torch.utils.data.random_split(trainset, [0.9, 0.1], generator = torch.Generator().manual_seed(515))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=NUM_WORKERS, collate_fn = collate_fn)
valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=NUM_WORKERS, collate_fn = collate_fn)

testset = COCODataset('pascalvoc', 'test')

testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=NUM_WORKERS, collate_fn = collate_fn)

batch = next(iter(trainloader))

# print(x[1])
# print(x[0])

# inp = x[0][0]
# lab = x[1][0]

# print(inp)
# print(lab)

In [4]:
toy = Resnext50(20)
resnet50 = LightningNet(Resnext50(20), 0.01)
guidedresnet50 = GuidedLightningNet(Resnext50(20), 0.01)

# print(toy(inp.unsqueeze(0)))
# print(predict(toy(inp.unsqueeze(0))))

# train model
# trainer = pl.Trainer(devices=1, accelerator="gpu", max_epochs = 60, callbacks=[LearningRateMonitor(logging_interval="step"), TQDMProgressBar(refresh_rate=10)])
# trainer.fit(resnet50, train_dataloaders = trainloader, val_dataloaders = valloader)

guidedtrainer = pl.Trainer(devices=4, accelerator="gpu", max_epochs = 60, callbacks=[LearningRateMonitor(logging_interval="step"), TQDMProgressBar(refresh_rate=10)])
guidedtrainer.fit(guidedresnet50, train_dataloaders = trainloader, val_dataloaders = valloader)

/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:263: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HP

Sanity Checking: 0it [00:00, ?it/s]

/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:208: UserWarning: num_workers>0, persistent_workers=False, and strategy=ddp_spawn may result in data loading bottlenecks. Consider setting persistent_workers=True (this is a limitation of Python .spawn() and PyTorch)
  rank_zero_warn(


Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]

/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(
/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:84: UserWarning: 

Sanity Checking DataLoader 0: 100%|██████████| 1/1 [00:05<00:00,  5.06s/it]

/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:536: PossibleUserWarning: It is recommended to use `self.log('val_class_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  warning_cache.warn(
/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:536: PossibleUserWarning: It is recommended to use `self.log('val_loc_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  warning_cache.warn(
/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:536: PossibleUserWarning: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across d

/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0:   0%|          | 0/10 [00:00<?, ?it/s] 

ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/strategies/launchers/multiprocessing.py", line 139, in _wrapping_function
    results = function(*args, **kwargs)
  File "/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py", line 650, in _fit_impl
    self._run(model, ckpt_path=self.ckpt_path)
  File "/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py", line 1112, in _run
    results = self._run_stage()
  File "/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py", line 1191, in _run_stage
    self._run_train()
  File "/home/nicholas/anacon

In [3]:
new_model =  LightningNet.load_from_checkpoint("/home/nicholas/summer_research_2023/guidedtraining/lightning_logs/version_16/checkpoints/epoch=59-step=2160.ckpt")

# trainer.test(new_model, testloader)

def model_wrapper(inputs, targets):
    output = new_model(inputs)
    # element-wise multiply outputs with one-hot encoded targets 
    # and compute sum of each row
    # This sums the prediction for all markers which exist in the cell
    return torch.sum(output * targets, dim=0)

x, y = batch[0], batch[1]
y_hat = new_model(x)
loss = nn.BCELoss()
classloss = loss(y_hat, y)
saliency = Saliency(model_wrapper)
attr = saliency.attribute(x, additional_forward_args=y)

print(attr.shape)

attr = attr.cpu().detach().numpy()

print(attr.shape)

locloss = batch_loc_loss(attr, batch[2])
aveloss = (classloss + locloss) / 2

print(aveloss)

/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:263: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  rank_zero_warn(
/home/nicholas/anaconda3/envs/cifar10/lib/python3.10/site-packages/captum/_utils/gradient.py:57: UserWarning: Input Tensor 0 did not already require gradients, required_grads has been set automatically.
  warnings.warn(


torch.Size([128, 3, 224, 224])
(128, 3, 224, 224)
tensor(0.3203, grad_fn=<DivBackward0>)
